In [1]:
from transformers import pipeline

question_answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [2]:
context =  """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = 'which deef learning libraries back Transformers?'
question_answerer(question = question, context = context)

{'score': 0.11953046917915344,
 'start': 78,
 'end': 106,
 'answer': 'Jax, PyTorch, and TensorFlow'}

In [3]:
long_context = """
🤗 Transformers: State of the Art NLP

🤗 Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction,
question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

🤗 Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and
then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and
can be modified to enable quick research experiments.

Why should I use transformers?

1. Easy-to-use state-of-the-art models:
  - High performance on NLU and NLG tasks.
  - Low barrier to entry for educators and practitioners.
  - Few user-facing abstractions with just three classes to learn.
  - A unified API for using all our pretrained models.
  - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
  - Practitioners can reduce compute time and production costs.
  - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model's lifetime:
  - Train state-of-the-art models in 3 lines of code.
  - Move a single model between TF2.0/PyTorch frameworks at will.
  - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
  - We provide examples for each architecture to reproduce the results published by its original authors.
  - Model internals are exposed as consistently as possible.
  - Model files can be used independently of the library for quick experiments.

🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""

In [4]:
question_answerer(question = question, context = long_context)

{'score': 0.19243748486042023,
 'start': 1847,
 'end': 1889,
 'answer': 'three most popular deep learning libraries'}

In [5]:
# now we will manually do the question answering
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [6]:
checkpoint = 'distilbert-base-cased-distilled-squad'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

c:\Users\Dawood Khan\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dawood Khan\.cache\huggingface\hub\models--distilbert-base-cased-distilled-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
inputs = tokenizer(question, context, return_tensors= 'pt')
outputs = model(**inputs)

In [8]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([1, 67]) torch.Size([1, 67])


In [10]:
import torch

sequence_ids = inputs.sequence_ids()
# Mask Everything apart from the tokens of the context
mask = [i !=1 for i in sequence_ids]
# Unmask the CLS Token as it is needed
mask[0] = False
mask = torch.tensor(mask)[None]

start_logits[mask] = -10000
end_logits[mask] = -10000

In [21]:
# Now we will apply softmax to turn logits in to probabilities
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)[0]

In [35]:
scores = start_probabilities[:,None] * end_probabilities[None,:]

In [36]:
print(scores)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 2.3735e-09, 2.0358e-09,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.3689e-09, 1.1741e-09,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], grad_fn=<MulBackward0>)


In [37]:
# mask the values where start_index > end_index by setting them to 0

scores = torch.triu(scores)

In [38]:
scores.shape

torch.Size([65, 65])

In [39]:
# Now we will get the index of the maximum
max_index = scores.argmax().item()
start_index = max_index//scores.shape[1]
end_index= max_index % scores.shape[1]

print(scores[start_index,end_index])

tensor(0.6609, grad_fn=<SelectBackward0>)


In [40]:
# now we need to convert character indices into context
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping = True)
offsets = inputs_with_offsets['offset_mapping']

start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]

In [41]:
result = {
    'answer': answer,
    'start':start_char,
    'end':end_char,
    'score':scores[start_index,end_index],
}
print(result)

{'answer': 'by the three most popular deep learning libraries — Jax, PyTorch, and Tensor', 'start': 26, 'end': 102, 'score': tensor(0.6609, grad_fn=<SelectBackward0>)}


In [42]:
# Now if we want top 5 Answers for our question
top_k= 5
flat_scores = scores.flatten()

In [43]:
# we will get the topindices and the top 5 socres
top_scores, top_indices = torch.topk(flat_scores, k = top_k)
# Convert the flat indices back to start and end indices
start_indices = (top_indices//scores.shape[1]).tolist()
end_indices = (top_indices%scores.shape[1]).tolist()

In [44]:
# Now we'll extract top 5 answers
answers = []
for s_idx,e_idx, score in zip(start_indices, end_indices, top_scores):
    start_char, _ = offsets[s_idx]
    _, end_char  = offsets[e_idx]
    answer = context[start_char:end_char]
    answers.append(
        {
            'answer': answer,
            'start':start_char,
            'end':end_char,
            'score':score.item(),
        }
    )



In [45]:
# Now sort the answers
answers  = sorted(answers, key = lambda x:x['score'], reverse = True)
print(answers)

[{'answer': 'by the three most popular deep learning libraries — Jax, PyTorch, and Tensor', 'start': 26, 'end': 102, 'score': 0.660897970199585}, {'answer': 'libraries — Jax, PyTorch, and Tensor', 'start': 66, 'end': 102, 'score': 0.11953002214431763}, {'answer': 'by', 'start': 26, 'end': 28, 'score': 0.040866415947675705}, {'answer': 'by the three most popular deep', 'start': 26, 'end': 56, 'score': 0.03716517984867096}, {'answer': 'backed by the three most popular deep learning libraries — Jax, PyTorch, and Tensor', 'start': 19, 'end': 102, 'score': 0.02173474244773388}]
